In [ ]:
import ast

from pathlib import Path

import requests
import pandas as pd

from jinja2 import Template
from bs4 import BeautifulSoup

from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference.models import SystemMessage, UserMessage

from ollama import chat
from ollama import ChatResponse


from settings import BASE_DIR, GITHUB_TOKEN

In [ ]:
job_descriptions = pd.read_excel("../data/bmw_jobs.xlsx")

In [ ]:
job_description = job_descriptions.loc[343, ('description')]

In [ ]:
application = Path(BASE_DIR / "data/application.txt").read_text()

In [ ]:
PROMPT = Template(Path(BASE_DIR / "data/rating.prompt.txt").read_text())

In [ ]:
response: ChatResponse = chat(
    model = 'llama3.2',
    messages=[
        {
            'role': 'user',
            'content': PROMPT.render(JOB_DESCRIPTION = job_description, APPLICATION = application),
        },
    ]
)

In [ ]:
response.message.content